In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Colab_data/hand_gesture_dataset.zip"
extract_path = "/content/hand_gesture_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [5]:
!pip install opencv-python matplotlib tensorflow


In [10]:
import os

def summarize_dataset_structure(root_dir):
    total_images = 0
    print(f"Scanning: {root_dir}\n")

    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)

        if os.path.isdir(folder_path):
            subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
            image_count = 0

            for sub in subfolders:
                sub_path = os.path.join(folder_path, sub)
                images = [f for f in os.listdir(sub_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                image_count += len(images)

            total_images += image_count
            print(f"📁 {folder}/ → {len(subfolders)} subfolders, {image_count} images")

    print(f"\n📸 Total images in dataset: {total_images}")

# Run it
summarize_dataset_structure("/content/hand_gesture_dataset")


Scanning: /content/hand_gesture_dataset

📁 leapgestrecog/ → 1 subfolders, 0 images
📁 leapGestRecog/ → 10 subfolders, 0 images

📸 Total images in dataset: 0


In [11]:
import os

sample_subfolder = "/content/hand_gesture_dataset/leapGestRecog"

# Look inside first subfolder
subfolders = sorted(os.listdir(sample_subfolder))
first_subfolder = os.path.join(sample_subfolder, subfolders[0])

print(f"Inspecting files in: {first_subfolder}\n")
file_list = os.listdir(first_subfolder)

for f in file_list[:10]:  # only show 10
    print(f)


Inspecting files in: /content/hand_gesture_dataset/leapGestRecog/00

09_c
04_fist_moved
05_thumb
10_down
06_index
08_palm_moved
07_ok
03_fist
02_l
01_palm


In [12]:
import os

gesture_root = "/content/hand_gesture_dataset/leapGestRecog"

total_images = 0
extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.pgm')

for subject in sorted(os.listdir(gesture_root)):
    subject_path = os.path.join(gesture_root, subject)
    if not os.path.isdir(subject_path):
        continue

    for gesture in sorted(os.listdir(subject_path)):
        gesture_path = os.path.join(subject_path, gesture)
        if not os.path.isdir(gesture_path):
            continue

        files = [f for f in os.listdir(gesture_path) if f.lower().endswith(extensions)]
        print(f"📁 {subject}/{gesture} → {len(files)} images")
        total_images += len(files)

print(f"\n📸 Total gesture images found: {total_images}")


📁 00/01_palm → 200 images
📁 00/02_l → 200 images
📁 00/03_fist → 200 images
📁 00/04_fist_moved → 200 images
📁 00/05_thumb → 200 images
📁 00/06_index → 200 images
📁 00/07_ok → 200 images
📁 00/08_palm_moved → 200 images
📁 00/09_c → 200 images
📁 00/10_down → 200 images
📁 01/01_palm → 200 images
📁 01/02_l → 200 images
📁 01/03_fist → 200 images
📁 01/04_fist_moved → 200 images
📁 01/05_thumb → 200 images
📁 01/06_index → 200 images
📁 01/07_ok → 200 images
📁 01/08_palm_moved → 200 images
📁 01/09_c → 200 images
📁 01/10_down → 200 images
📁 02/01_palm → 200 images
📁 02/02_l → 200 images
📁 02/03_fist → 200 images
📁 02/04_fist_moved → 200 images
📁 02/05_thumb → 200 images
📁 02/06_index → 200 images
📁 02/07_ok → 200 images
📁 02/08_palm_moved → 200 images
📁 02/09_c → 200 images
📁 02/10_down → 200 images
📁 03/01_palm → 200 images
📁 03/02_l → 200 images
📁 03/03_fist → 200 images
📁 03/04_fist_moved → 200 images
📁 03/05_thumb → 200 images
📁 03/06_index → 200 images
📁 03/07_ok → 200 images
📁 03/08_palm_move

In [24]:
import os
import shutil

source_root = "/content/hand_gesture_dataset/leapGestRecog"
destination_root = "/content/final_dataset"

# Create destination folders
gesture_labels = {
    "01_palm": "palm",
    "02_l": "l",
    "03_fist": "fist",
    "04_fist_moved": "fist_moved",
    "05_thumb": "thumb",
    "06_index": "index",
    "07_ok": "ok",
    "08_palm_moved": "palm_moved",
    "09_c": "c",
    "10_down": "down"
}

os.makedirs(destination_root, exist_ok=True)
for label in gesture_labels.values():
    os.makedirs(os.path.join(destination_root, label), exist_ok=True)

# Copy and rename files
for subject in os.listdir(source_root):
    subject_path = os.path.join(source_root, subject)
    if not os.path.isdir(subject_path):
        continue

    for gesture_folder in os.listdir(subject_path):
        gesture_path = os.path.join(subject_path, gesture_folder)
        if not os.path.isdir(gesture_path):
            continue

        gesture_label = gesture_labels.get(gesture_folder)
        if gesture_label is None:
            continue  # skip unknown folders

        dest_folder = os.path.join(destination_root, gesture_label)

        for i, file in enumerate(os.listdir(gesture_path)):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.pgm')):
                src = os.path.join(gesture_path, file)
                dst = os.path.join(dest_folder, f"{gesture_label}_{subject}_{i:04d}.png")
                shutil.copy(src, dst)

print("✅ Dataset flattened and saved to /content/final_dataset/")


✅ Dataset flattened and saved to /content/final_dataset/


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 64  # You can also try 128 for higher resolution

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/final_dataset',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    '/content/final_dataset',
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Found 30400 images belonging to 12 classes.
Found 7600 images belonging to 12 classes.


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 684,490 (2.61 MB)

 Trainable params: 684,490 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # you can increase to 15–20 later
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 50s 89ms/step - accuracy: 0.6935 - loss: 0.8647 - val_accuracy: 0.9212 - val_loss: 0.3462
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 0.9886 - loss: 0.0380 - val_accuracy: 0.9273 - val_loss: 0.4171
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 45s 90ms/step - accuracy: 0.9901 - loss: 0.0308 - val_accuracy: 0.9440 - val_loss: 0.1299
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 42s 85ms/step - accuracy: 0.9964 - loss: 0.0122 - val_accuracy: 0.9643 - val_loss: 0.1271
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - accuracy: 0.9957 - loss: 0.0117 - val_accuracy: 0.9720 - val_loss: 0.1006
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 42s 84ms/step - accuracy: 0.9956 - loss: 0.0121 - val_accuracy: 0.9187 - val_loss: 0.2984
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 0.9976 - loss: 0.0079 - val_accuracy: 0.8882 - val_loss: 0.6673
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 43s 86ms/step - accuracy: 0.9961 - loss: 0.0152 - 

In [17]:
# Path to save model in Google Drive
model_save_path = '/content/drive/MyDrive/gesture_model.h5'

# Save the trained model
model.save(model_save_path)

print(f"✅ Model saved to: {model_save_path}")


✅ Model saved to: /content/drive/MyDrive/gesture_model.h5
